# Force field example

## low level interface

To show how the components of NFFLr work together, let's train a formation energy model using the `mlearn` dataset.
We can use the `periodic_radius_graph` transform to configure the `AtomsDataset` to automatically transform atomic configurations into `DGLGraph`s.

In [1]:
import nfflr

rcut = 4.0
transform = nfflr.nn.PeriodicRadiusGraph(cutoff=rcut)
dataset = nfflr.data.mlearn_dataset("Si", transform=transform)
dataset[0]

/Users/bld/projects/nfflr/nfflr/atoms.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.positions = torch.tensor(positions, dtype=dtype)


(Graph(num_nodes=63, num_edges=838,
       ndata_schemes={'coord': Scheme(shape=(3,), dtype=torch.float32), 'atomic_number': Scheme(shape=(), dtype=torch.int32)}
       edata_schemes={'r': Scheme(shape=(3,), dtype=torch.float32)}),
 {'energy': tensor(-295.4975),
  'forces': tensor([[-2.9819e+00,  6.0692e-01,  3.4814e+00],
          [ 1.0296e+00, -1.3696e-01, -2.8213e-01],
          [ 1.2912e+00,  2.5037e+00,  1.8027e-01],
          [-5.0030e-01, -6.8284e-01, -2.6741e+00],
          [-1.2477e+00,  5.4527e-01,  4.5055e-01],
          [ 2.5866e-01, -1.2084e+00,  1.4178e+00],
          [ 1.0119e+00, -9.7490e-01,  1.3844e+00],
          [-1.1932e+00,  1.5503e+00, -2.9724e-01],
          [ 9.7073e-01,  9.3589e-01,  5.8628e-03],
          [-1.9438e-01,  7.4742e-01,  3.8357e-01],
          [ 8.0306e-01, -4.9752e-01,  1.7991e+00],
          [-2.0877e-01,  2.1184e-01,  1.6049e-03],
          [ 1.1246e-01,  1.1640e-01,  2.1472e-02],
          [-1.0178e+00, -3.9186e-02, -1.9705e-01],
          [-1

Set up a medium-sized ALIGNN model, with atomic reference energies estimated from the training set:

In [2]:
from nfflr.models.gnn import alignn

reference_energies = dataset.estimate_reference_energies()

cfg = nfflr.models.ALIGNNConfig(
    transform=transform,
    cutoff=nfflr.nn.Cosine(rcut),
    reference_energies=reference_energies,
    alignn_layers=1, 
    gcn_layers=2, 
    embedding_features=16,
    edge_input_features=16,
    triplet_input_features=16,
    hidden_features=32,
    norm="layernorm", 
    atom_features="embedding",
    compute_forces=True,
)
model = nfflr.models.ALIGNN(cfg)

atoms, target = dataset[0]
model(atoms)

/Users/bld/.pyenv/versions/3.10.9/envs/nfflr/lib/python3.10/site-packages/dgl/backend/pytorch/tensor.py:445: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


{'energy': tensor(0.4190, grad_fn=<SqueezeBackward0>),
 'forces': tensor([[ 4.9852e-07, -1.3226e-07, -1.6838e-07],
         [-2.6099e-07,  6.0730e-07, -1.0012e-07],
         [-1.5601e-07,  1.8643e-07,  5.4123e-07],
         [ 4.9777e-08, -4.2036e-08,  3.0614e-07],
         [ 2.6156e-07,  2.6011e-08, -2.2906e-07],
         [-2.5958e-07,  2.4926e-08, -1.5123e-07],
         [-6.6287e-07,  3.5392e-08, -3.1768e-07],
         [ 2.0246e-07, -1.4481e-07,  1.3680e-07],
         [-3.6468e-07, -7.8529e-08, -2.0065e-07],
         [-1.6847e-07, -5.5197e-07, -3.0397e-07],
         [-4.4315e-07,  1.7318e-07, -2.7184e-07],
         [-3.9167e-07, -4.3728e-07,  2.4780e-07],
         [-1.0396e-07,  2.3536e-07,  3.7972e-07],
         [ 1.0582e-07,  4.1403e-07,  9.9467e-08],
         [ 1.7119e-07, -4.0077e-07,  1.0163e-07],
         [-2.9991e-07,  1.8739e-07, -2.7835e-07],
         [-1.6495e-07,  2.9263e-07, -4.5218e-07],
         [-2.7957e-07, -3.9808e-08,  1.2410e-07],
         [ 1.7630e-07,  1.5395e-07,

In [3]:
import numpy as np

import torch
from torch import nn
from torch.utils.data import DataLoader, SubsetRandomSampler

batchsize = 2

train_loader = DataLoader(
    dataset,
    batch_size=batchsize, 
    collate_fn=dataset.collate, 
    sampler=SubsetRandomSampler(dataset.split["train"]),
    drop_last=True
)
next(iter(train_loader))

(Graph(num_nodes=128, num_edges=1862,
       ndata_schemes={'coord': Scheme(shape=(3,), dtype=torch.float32), 'atomic_number': Scheme(shape=(), dtype=torch.int32)}
       edata_schemes={'r': Scheme(shape=(3,), dtype=torch.float32)}),
 {'energy': tensor([-339.0587, -346.9658]),
  'n_atoms': tensor([64, 64]),
  'forces': tensor([[-0.6067,  0.4133, -1.2317],
          [ 1.1389, -2.1391, -1.2522],
          [ 0.9383,  1.2495, -0.7030],
          [-0.9375,  1.4090, -1.8885],
          [ 1.9405, -1.5400, -1.0583],
          [-0.7696, -0.0420,  0.8229],
          [ 1.1848, -0.4141, -1.3028],
          [ 0.2667,  1.0219,  0.3032],
          [ 0.3974,  0.8765,  0.6913],
          [-1.7408,  0.0886, -0.0915],
          [-1.9527,  1.7243,  0.3515],
          [ 0.4534,  1.4929, -0.4008],
          [ 0.1687, -0.3284,  1.3564],
          [ 0.0762, -0.4296,  0.0637],
          [-0.3450,  0.2660,  0.0698],
          [-0.3287, -0.7818,  2.3715],
          [-1.1107, -2.1853,  0.5160],
          [ 0.5564

Now we can set up a PyTorch optimizer and objective function and optimize the model parameters with an explicit training loop. See the [PyTorch quickstart tutorial for more context)[https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html].

For force field training, we use a custom loss function since the output of the model is structured:

In [4]:
from tqdm import tqdm

criteria = {"energy": nn.MSELoss(), "forces": nn.HuberLoss(delta=5.0)}

def ff_criterion(outputs, targets):
    """Specify combined energy and force loss."""

    n_atoms = targets["n_atoms"]

    # scale loss by crystal size
    energy_loss = criteria["energy"](
        outputs["energy"] / n_atoms, targets["energy"] / n_atoms
    )

    # # scale the forces before the loss
    force_scale = 1.0
    force_loss = criteria["forces"](outputs["forces"], targets["forces"])

    return energy_loss + force_scale * force_loss

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.1)

training_loss = []
for epoch in range(5):
    for step, (g, y) in enumerate(tqdm(train_loader)):
        pred = model(g)
        loss = ff_criterion(pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        training_loss.append(loss.item())

  0%|          | 0/107 [00:00<?, ?it/s]/Users/bld/.pyenv/versions/3.10.9/envs/nfflr/lib/python3.10/site-packages/dgl/backend/pytorch/tensor.py:445: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (
100%|██████████| 107/107 [00:11<00:00,  9.33it/s]


# using the ignite-based NFFLr trainer

In [5]:
import tempfile
from nfflr import train

In [6]:
rank = 0
training_config = {
    "dataset": dataset,
    "model": model,
    "optimizer": optimizer,
    "criterion": ff_criterion,
    "random_seed": 42,
    "batch_size": 2,
    "learning_rate": 1e-3,
    "weight_decay": 0.1,
    "epochs": 5,
    "warmup_steps": 100,
    "num_workers": 0,
    "progress": True,
    "output_dir": tempfile.TemporaryDirectory().name
}
train.run_train(rank, training_config)

2024-02-15 09:25:24,547 ignite.distributed.auto.auto_dataloader INFO: Use data loader kwargs for dataset '<nfflr.data.dataset.': 
	{'collate_fn': <function AtomsDataset.collate_forcefield at 0x2a11ffa30>, 'batch_size': 2, 'sampler': <torch.utils.data.sampler.SubsetRandomSampler object at 0x1092a5f30>, 'drop_last': True, 'num_workers': 0, 'pin_memory': False}
2024-02-15 09:25:24,547 ignite.distributed.auto.auto_dataloader INFO: Use data loader kwargs for dataset '<nfflr.data.dataset.': 
	{'collate_fn': <function AtomsDataset.collate_forcefield at 0x2a11ffa30>, 'batch_size': 2, 'sampler': <torch.utils.data.sampler.SubsetRandomSampler object at 0x2a21214e0>, 'drop_last': False, 'num_workers': 0, 'pin_memory': False}


starting training loop


/Users/bld/.pyenv/versions/3.10.9/envs/nfflr/lib/python3.10/site-packages/dgl/backend/pytorch/tensor.py:445: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


[1/107]   1%|           [00:00<?]

[1/10]  10%|#          [00:00<?]

...

[1/10]  10%|#          [00:00<?]

train results - Epoch: 5  Avg loss: 8.32
energy: 172.06  force: 0.4186


[1/13]   8%|7          [00:00<?]

val results - Epoch: 5  Avg loss: 7.36
energy: 165.02  force: 0.4125


7.359944857083834